In [1]:
#pip install duckdb
#pip install pyarrow

In [2]:
#importar bibliotecas
import duckdb as db
import pyarrow as pa
import pyarrow.parquet as pq

In [3]:
#importar base de dados
stock_data = pq.read_table(r"C:\Users\Maria\Desktop\projetos\all_stock_data.parquet")

In [4]:
print(stock_data.schema)

Date: date32[day]
Ticker: string
Open: double
High: double
Low: double
Close: double
Volume: double
Dividends: double
Stock Splits: double
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 1276


In [5]:
#conectar base de dados com duckdb
conn_stock = db.connect()
conn_stock.register('stock_data', stock_data)

In [6]:
# visualizar tabela
query = '''
SELECT * FROM stock_data
LIMIT 10
'''
conn_stock.sql( query )

┌────────────┬─────────┬────────┬─────────────────────┬─────────────────────┬─────────────────────┬───────────┬───────────┬──────────────┐
│    Date    │ Ticker  │  Open  │        High         │         Low         │        Close        │  Volume   │ Dividends │ Stock Splits │
│    date    │ varchar │ double │       double        │       double        │       double        │  double   │  double   │    double    │
├────────────┼─────────┼────────┼─────────────────────┼─────────────────────┼─────────────────────┼───────────┼───────────┼──────────────┤
│ 1962-01-02 │ ED      │    0.0 │  0.2658275556233194 │ 0.26178762316703796 │ 0.26178762316703796 │   25600.0 │       0.0 │          0.0 │
│ 1962-01-02 │ CVX     │    0.0 │ 0.04680890217423439 │ 0.04606926600933256 │ 0.04680890217423439 │  105840.0 │       0.0 │          0.0 │
│ 1962-01-02 │ GD      │    0.0 │ 0.21003275954390174 │ 0.20306070787008793 │  0.2082897424697876 │ 2648000.0 │       0.0 │          0.0 │
│ 1962-01-02 │ BP      │   

análise descritiva

In [7]:
# quantidade de linhas
query = '''
SELECT COUNT(*) AS quantidade_linhas
FROM stock_data
'''
conn_stock.sql( query )

┌───────────────────┐
│ quantidade_linhas │
│       int64       │
├───────────────────┤
│          34646258 │
└───────────────────┘

In [8]:
# Contagem de ações distintas
query = '''
SELECT  COUNT(DISTINCT Ticker) AS quantidade_tickets
FROM stock_data
'''
conn_stock.sql( query)

┌────────────────────┐
│ quantidade_tickets │
│       int64        │
├────────────────────┤
│               9315 │
└────────────────────┘

In [9]:
#primeiro registro da base de dados
query_primeiro = '''
SELECT MIN(Date) AS primeiro_registro
FROM stock_data
'''

#ultimo registro
query_ultimo = '''
SELECT MAX(Date) AS ultimo_registro
FROM stock_data
'''
print(conn_stock.sql( query_primeiro ))
print(conn_stock.sql( query_ultimo ))

┌───────────────────┐
│ primeiro_registro │
│       date        │
├───────────────────┤
│ 1962-01-02        │
└───────────────────┘

┌─────────────────┐
│ ultimo_registro │
│      date       │
├─────────────────┤
│ 2024-11-04      │
└─────────────────┘



In [10]:
#preço de abertura médio por ticket
query = '''
SELECT  
Ticker,
AVG (Open) AS média_preço_de_abertura
FROM stock_data
GROUP BY Ticker
'''
conn_stock.sql(query)

┌─────────┬─────────────────────────┐
│ Ticker  │ média_preço_de_abertura │
│ varchar │         double          │
├─────────┼─────────────────────────┤
│ OTTR    │      14.782950563737534 │
│ UFI     │       24.94066507042611 │
│ BCE     │      14.863620216852917 │
│ ALK     │       18.27544942692912 │
│ CHE     │        83.0589628537036 │
│ MCS     │        9.02190887100567 │
│ NJR     │      12.318776128480746 │
│ IMO     │      16.473297380857286 │
│ PNW     │      22.568527685423447 │
│ HST     │       8.022232580623175 │
│  ·      │               ·         │
│  ·      │               ·         │
│  ·      │               ·         │
│ SBXD-UN │      10.053905521739614 │
│ RITR    │       5.363333323422601 │
│ MBBC    │       8.902771055698395 │
│ BSIIU   │        9.96791660785675 │
│ SNYR    │       8.179375112056732 │
│ SPHL    │       4.421749989191691 │
│ TDTH    │      3.2892000019550323 │
│ KHOB    │      0.8674468081048194 │
│ SBXD    │      10.003000020980835 │
│ CHKEZ   │ 

In [11]:
#maior preço de fechamento já registrado
query = '''
SELECT
Ticker, Close
FROM 
stock_data
WHERE Close = (
    SELECT MAX(Close)
    FROM stock_data)
LIMIT 1
'''
conn_stock.sql( query )

┌─────────┬────────────────────────┐
│ Ticker  │         Close          │
│ varchar │         double         │
├─────────┼────────────────────────┤
│ PTPIF   │ 1.5072897744279783e+28 │
└─────────┴────────────────────────┘

análise de performance

In [12]:
# retorno diário médio por ticker
query = '''
SELECT
Ticker, 
AVG(Close - Open) AS retorno_diário_médio
FROM stock_data
GROUP BY Ticker
'''
conn_stock.sql( query )

┌─────────┬────────────────────────┐
│ Ticker  │  retorno_diário_médio  │
│ varchar │         double         │
├─────────┼────────────────────────┤
│ ERIC    │     0.0613247656180622 │
│ ETR     │   0.005506242669400805 │
│ FUJIY   │    0.12022634423065129 │
│ LNT     │     0.2774411340861314 │
│ PH      │    0.29494062212509986 │
│ CBSH    │   0.024801070559811005 │
│ FDX     │    0.01279052252444778 │
│ PHM     │      0.129586121940654 │
│ NFG     │ -0.0002783340416960897 │
│ TR      │    0.13878150783299573 │
│ ·       │             ·          │
│ ·       │             ·          │
│ ·       │             ·          │
│ HCWC    │    -0.1586944427755144 │
│ ETHW    │   -0.09427028088956266 │
│ JAGL    │                    0.0 │
│ MANA    │   -0.21106024248054228 │
│ IDXG    │    0.03255422431302358 │
│ MGSD    │   -0.08112833835184574 │
│ BLFBY   │   -0.06500005722045898 │
│ GRAF-UN │ -0.0017699432373046876 │
│ INR     │                    0.0 │
│ SHCRW   │                    0.0 │
├

In [13]:
# Ticker de maior volume total
query = '''
SELECT 
Ticker, 
SUM(Volume) AS volume_total
FROM stock_data
GROUP BY Ticker
ORDER BY volume_total DESC
LIMIT 1
'''
conn_stock.sql( query )

┌─────────┬─────────────────┐
│ Ticker  │  volume_total   │
│ varchar │     double      │
├─────────┼─────────────────┤
│ NVDA    │ 3912343141792.0 │
└─────────┴─────────────────┘

In [14]:
#maior dividendo pago por ticker
query = '''
SELECT 
Ticker, 
MAX(Dividends) AS maior_dividendo
FROM stock_data
GROUP BY Ticker
ORDER BY maior_dividendo DESC
'''
conn_stock.sql( query )

┌─────────┬─────────────────┐
│ Ticker  │ maior_dividendo │
│ varchar │     double      │
├─────────┼─────────────────┤
│ OSCUF   │          4500.0 │
│ NTTDF   │          3000.0 │
│ TOPS    │          1000.0 │
│ PSHG    │          1000.0 │
│ NURO    │          1000.0 │
│ VHI     │          1000.0 │
│ SHIP    │          1000.0 │
│ TOELF   │           562.0 │
│ GYRO    │       509.55557 │
│ GLBS    │           500.0 │
│  ·      │              ·  │
│  ·      │              ·  │
│  ·      │              ·  │
│ STSR    │             0.0 │
│ AMTM    │             0.0 │
│ SKK     │             0.0 │
│ SOBO    │             0.0 │
│ SEPN    │             0.0 │
│ PMAX    │             0.0 │
│ EBFI    │             0.0 │
│ WSTRF   │             0.0 │
│ TRLI    │             0.0 │
│ TRIC    │             0.0 │
├─────────┴─────────────────┤
│   9315 rows (20 shown)    │
└───────────────────────────┘

Análise temporal

In [15]:
#fechamento médio de Ticker por mês
query = '''
SELECT
Ticker,
MONTH(Date) AS mês, 
AVG(Close) AS média_fechamento
FROM stock_data
GROUP BY Ticker, mês
ORDER BY mês ASC
'''
conn_stock.sql( query )

┌─────────┬───────┬────────────────────┐
│ Ticker  │  mês  │  média_fechamento  │
│ varchar │ int64 │       double       │
├─────────┼───────┼────────────────────┤
│ AIG     │     1 │ 228.23548918503982 │
│ GWW     │     1 │  99.16729627191907 │
│ HWKN    │     1 │  9.463538922377694 │
│ ABEO    │     1 │   519974.709779277 │
│ ADX     │     1 │  4.672487198715042 │
│ CIA     │     1 │  5.345876436249292 │
│ EQT     │     1 │ 12.831086936931495 │
│ FELE    │     1 │ 19.366807707927325 │
│ ATRO    │     1 │  7.396897223084967 │
│ UNP     │     1 │   44.5187125150816 │
│  ·      │     · │           ·        │
│  ·      │     · │           ·        │
│  ·      │     · │           ·        │
│ MRK     │     2 │  19.71883352131121 │
│ HP      │     2 │ 18.195945392146495 │
│ AVA     │     2 │ 12.353978851582946 │
│ TER     │     2 │ 21.871247902633698 │
│ CINF    │     2 │    29.052474413226 │
│ INSI    │     2 │  7.760293339666032 │
│ PHM     │     2 │ 15.395265645113602 │
│ HST     │     

In [16]:
#em quais dias os preços do fechamento ultrapassaram os preços de abertura em 5%
query = '''
SELECT 
Date,
Ticker
FROM stock_data
WHERE (Close - Open)/Open > 0.05
'''
conn_stock.sql( query )

┌──────────────┬─────────┐
│     Date     │ Ticker  │
│     date     │ varchar │
├──────────────┼─────────┤
│ 1962-01-02   │ ED      │
│ 1962-01-02   │ CVX     │
│ 1962-01-02   │ GD      │
│ 1962-01-02   │ BP      │
│ 1962-01-02   │ MSI     │
│ 1962-01-02   │ HON     │
│ 1962-01-02   │ FL      │
│ 1962-01-02   │ GT      │
│ 1962-01-02   │ JNJ     │
│ 1962-01-02   │ MMM     │
│     ·        │  ·      │
│     ·        │  ·      │
│     ·        │  ·      │
│ 1963-10-22   │ DTE     │
│ 1963-10-22   │ ED      │
│ 1963-10-22   │ IP      │
│ 1963-10-22   │ RTX     │
│ 1963-10-22   │ MRK     │
│ 1963-10-22   │ AEP     │
│ 1963-10-22   │ CVX     │
│ 1963-10-22   │ PG      │
│ 1963-10-22   │ FL      │
│ 1963-10-22   │ GD      │
├──────────────┴─────────┤
│ ? rows       2 columns │
└────────────────────────┘

In [17]:
#volume médio negociado por dia da semana
query = '''
SELECT 
Ticker,
strftime('%A', Date) AS dia_da_semana, 
AVG(Volume) AS volume_médio
FROM
stock_data
GROUP BY Ticker, dia_da_semana
'''
conn_stock.sql( query )

┌─────────┬───────────────┬────────────────────┐
│ Ticker  │ dia_da_semana │    volume_médio    │
│ varchar │    varchar    │       double       │
├─────────┼───────────────┼────────────────────┤
│ FUL     │ Wednesday     │ 269752.20808383235 │
│ MMM     │ Wednesday     │   2388361.19081601 │
│ APA     │ Wednesday     │ 2773221.9178723404 │
│ VMI     │ Wednesday     │  99921.03293413174 │
│ GATX    │ Wednesday     │ 217315.94311377246 │
│ AEM     │ Wednesday     │  964285.4416167665 │
│ TER     │ Wednesday     │ 1846107.0733532934 │
│ WDFC    │ Wednesday     │ 51282.562946260805 │
│ ASA     │ Wednesday     │ 205657.53820735234 │
│ HPQ     │ Wednesday     │  15291588.14334471 │
│  ·      │   ·           │               ·    │
│  ·      │   ·           │               ·    │
│  ·      │   ·           │               ·    │
│ YXT     │ Monday        │            79140.0 │
│ GRAF    │ Monday        │           171120.0 │
│ TCBC    │ Monday        │             2470.0 │
│ HONDU   │ Monday  

comparações

In [18]:
#ticker com maior preço de fechamento médio durante o período
query = '''
SELECT
Ticker,
AVG(Close) AS média_fechamento
FROM
stock_data
GROUP BY Ticker
ORDER BY média_fechamento DESC
LIMIT 1
'''
conn_stock.sql( query )

┌─────────┬────────────────────────┐
│ Ticker  │    média_fechamento    │
│ varchar │         double         │
├─────────┼────────────────────────┤
│ PTPIF   │ 1.0797134821793699e+27 │
└─────────┴────────────────────────┘

In [19]:
#comparar o volume médio entre dois ticker aleatórios
query = '''
SELECT
Ticker,
AVG(Volume) AS volume_médio
FROM stock_data
WHERE Ticker = 'PTPIF' OR Ticker = 'USLM'
GROUP BY Ticker
'''
conn_stock.sql( query )

┌─────────┬────────────────────┐
│ Ticker  │    volume_médio    │
│ varchar │       double       │
├─────────┼────────────────────┤
│ PTPIF   │  1616.056456120738 │
│ USLM    │ 31596.555269237604 │
└─────────┴────────────────────┘

In [ ]:
#ticker que teve o maior crescimento percentual de fechamento registrado de um dia para o outro
query = '''
SELECT Ticker
From (
SELECT
Ticker,
(Close - LAG(Close) OVER(ORDER BY Date))/
LAG(Close) OVER(ORDER BY Date) AS diferença_percentual
FROM stock_data
)
ORDER BY diferença_percentual DESC
LIMIT 1
'''
conn_stock.sql( query )

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────┐
│ Ticker  │
│ varchar │
├─────────┤
│ DLPN    │
└─────────┘